In [1]:
import pandas as pd
from google.cloud import storage
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
games_df = pd.read_csv('data/games.csv')
plays_df = pd.read_csv('data/plays.csv')
players_df = pd.read_csv('data/players.csv')
player_stat_df = pd.read_csv('data/player_play.csv')

In [3]:
def identify_attached_tes(game_id, play_id, tracking_df, players_df, y_threshold=1.75, x_threshold=0.35):
    """
    Identify TEs who are attached to tackles using stricter y_threshold
    """
    play_frames = tracking_df[
        (tracking_df['gameId'] == game_id) & 
        (tracking_df['playId'] == play_id)
    ]
    
    # Check if we have any snap frames
    snap_frames = play_frames[play_frames['event'] == 'ball_snap']
    if len(snap_frames) == 0:
        return pd.DataFrame()  # Return empty DataFrame if no snap
        
    snap_frame = snap_frames['frameId'].iloc[0]
    play_direction = play_frames['playDirection'].iloc[0]
    
    # Get positions at snap
    positions = play_frames[play_frames['frameId'] == snap_frame]
    
    # Get ball position at snap
    ball_position = positions[positions['nflId'].isna()].iloc[0]  # Ball rows have NaN for nflId
    ball_y = ball_position['y']
    
    tackle_positions = positions[
        positions['nflId'].isin(
            players_df[players_df['position'] == 'T']['nflId']
        )
    ]
    
    te_positions = positions[
        positions['nflId'].isin(
            players_df[players_df['position'] == 'TE']['nflId']
        )
    ]
    
    attached_tes = []
    for _, te in te_positions.iterrows():
        x_dist, y_dist = calculate_xy_distances_from_tackles(te, tackle_positions, play_direction)
        if abs(y_dist) <= y_threshold and abs(x_dist) <= x_threshold:
            attached_tes.append({
                'nflId': te['nflId'],
                'displayName': te['displayName'],
                'y_dist': y_dist,  # Keep original signed values for reference
                'x_dist': x_dist,  # Keep original signed values for reference
                'abs_y_dist': abs(y_dist),  # Add absolute values
                'abs_x_dist': abs(x_dist), 
                'playDirection': play_direction,
                'ball_y': ball_y
            })
    
    return pd.DataFrame(attached_tes)

In [4]:
def calculate_xy_distances_from_tackles(te_row, tackle_positions, play_direction):
    """
    Calculate distances between TE and nearest tackle
    Now correctly handling left/right side determination based on play direction
    """
    te_x = te_row['x']
    te_y = te_row['y']
    
    # Sort tackles by y-value
    sorted_tackles = tackle_positions.sort_values('y')
    high_y_tackle = sorted_tackles.iloc[-1]
    low_y_tackle = sorted_tackles.iloc[0]
    
    # When going left, everything is flipped
    # Lower y-value is on the left side of the formation
    # Higher y-value is on the right side of the formation
    if play_direction == 'left':
        left_tackle = low_y_tackle
        right_tackle = high_y_tackle
    else:  # play_direction == 'right'
        left_tackle = high_y_tackle
        right_tackle = low_y_tackle
    
    # Calculate distances to both tackles
    lt_x_dist = te_x - left_tackle['x']
    lt_y_dist = te_y - left_tackle['y']
    rt_x_dist = te_x - right_tackle['x']
    rt_y_dist = te_y - right_tackle['y']
    
    # Find nearest tackle and calculate distance
    if abs(lt_y_dist) < abs(rt_y_dist):
        x_dist = lt_x_dist
        y_dist = lt_y_dist
    else:
        x_dist = rt_x_dist
        y_dist = rt_y_dist
    
    return x_dist, y_dist

In [5]:
def create_game_phase(df):
    """
    Create game phase feature from quarter and game clock
    """
    conditions = [
        (df['quarter'] <= 2),
        (df['quarter'] == 3),
        (df['quarter'] == 4) & (df['gameClock'] > "5:00"),
        (df['quarter'] == 4) & (df['gameClock'] <= "5:00")
    ]
    choices = ['early_game', 'third_quarter', 'early_fourth', 'crunch_time']
    df['game_phase'] = np.select(conditions, choices, default='unknown')
    return df

def create_score_situation(df):
    """
    Create score situation feature from score differential
    Points are from perspective of home team (home - visitor)
    0-8 points = 1 score game (TD + potential 2pt conversion)
    9-16 points = 2 score game
    17+ points = big deficit/lead
    """
    df['score_differential'] = df['preSnapHomeScore'] - df['preSnapVisitorScore']
    df['score_situation'] = pd.cut(
        df['score_differential'],
        bins=[-100, -17, -9, -3, 3, 9, 17, 100],
        labels=['down_big', 'down_2scores', 'down_1score', 'close', 'up_1score', 'up_2scores', 'up_big']
    )
    return df

def clean_motion_features(df):
    """
    Ensure motion features are boolean
    """
    motion_cols = ['inMotionAtBallSnap', 'shiftSinceLineset', 'motionSinceLineset']
    for col in motion_cols:
        df[col] = df[col].fillna(False).astype(bool)
    return df

def create_play_features(df):
    """
    Clean up and simplify play design features
    """
    # Simplify formation
    df['formation_simple'] = df['offenseFormation'].map({
        'SHOTGUN': 'shotgun',
        'SINGLEBACK': 'singleback',
        'I_FORM': 'i_form',
        'PISTOL': 'pistol',
        'JUMBO': 'jumbo',
        'EMPTY': 'empty'
    }).fillna('other')
    
    # Simplify dropback types
    df['dropback_simple'] = df['dropbackType'].map({
        'TRADITIONAL': 'traditional',
        'DESIGNED_ROLLOUT_LEFT': 'rollout',
        'DESIGNED_ROLLOUT_RIGHT': 'rollout',
        'SCRAMBLE': 'scramble',
        'SCRAMBLE_ROLLOUT_LEFT': 'scramble',
        'SCRAMBLE_ROLLOUT_RIGHT': 'scramble'
    }).fillna('other')
    
    return df

def create_all_features(df):
    """
    Apply all feature engineering steps
    """
    df = create_score_situation(df)
    df = create_game_phase(df)
    df = clean_motion_features(df)
    df = create_play_features(df)
    return df

In [6]:
def process_week(tracking_df, plays_df, players_df, player_play_df):
    """
    Process a single week of data to identify attached TEs and their routes
    """
    # Get all unique games
    games = tracking_df['gameId'].unique()
    
    all_attached_tes = []
    
    for game_id in games:
        # Get all plays for this game
        game_plays = tracking_df[tracking_df['gameId'] == game_id]['playId'].unique()
        
        for play_id in game_plays:
            attached = identify_attached_tes(game_id, play_id, tracking_df, players_df)
            if len(attached) > 0:
                attached['gameId'] = game_id
                attached['playId'] = play_id
                all_attached_tes.append(attached)
    
    if not all_attached_tes:
        return pd.DataFrame()
        
    # Combine all results
    te_play_info = pd.concat(all_attached_tes, ignore_index=True)
    
    # Add necessary columns from plays_df and player_play_df
    te_play_info = te_play_info.merge(
        plays_df[['gameId', 'playId', 'isDropback', 'passResult', 'playAction', 
                 'offenseFormation', 'down', 'yardsToGo', 'absoluteYardlineNumber',
                 'pff_passCoverage', 'pff_runConceptPrimary', 'preSnapHomeScore',
                 'preSnapVisitorScore', 'quarter', 'gameClock', 'pff_manZone', 'dropbackType']], 
        on=['gameId', 'playId']
    )
    
    te_play_info = te_play_info.merge(
        player_play_df[['gameId', 'playId', 'nflId', 'routeRan', 
                       'inMotionAtBallSnap', 'shiftSinceLineset', 'motionSinceLineset']], 
        on=['gameId', 'playId', 'nflId']
    )
    
    # Create features
    te_play_info = create_all_features(te_play_info)
    
    return te_play_info

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
client = storage.Client()
bucket_name = 'nfl_databowl_bucket'
bucket = client.bucket(bucket_name)

In [9]:
def download_from_bucket(blob_name, output_file):
    blob = bucket.blob(blob_name)
    blob.download_to_filename(output_file)
    print(f"Downloaded {blob_name} to {output_file}")

In [10]:
os.makedirs('data', exist_ok=True)

In [11]:
files_to_download = [
    'games.csv',
    'plays.csv',
    'players.csv',
    'player_play.csv',
    'tracking_week_1.csv',
    'tracking_week_2.csv',
    'tracking_week_3.csv',
    'tracking_week_4.csv',
    'tracking_week_5.csv',
    'tracking_week_6.csv',
    'tracking_week_7.csv',
    'tracking_week_8.csv',
    'tracking_week_9.csv',
]

In [12]:
for file in files_to_download:
    local_file = f'data/{file}'
    if not os.path.exists(local_file):
        download_from_bucket(file, local_file)

In [13]:
def read_tracking_data(file_path, chunksize=100000):
    """
    Read large tracking data files in chunks
    """
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        chunks.append(chunk)
    return pd.concat(chunks)

In [97]:
all_weeks_data = []
for week in range(1, 10):  # Adjust range based on available weeks
    print(f"Processing week {week}...")
    
    # Load tracking data for week using chunking
    tracking_df = read_tracking_data(f'data/tracking_week_{week}.csv')
    
    # Process week
    week_data = process_week(tracking_df, plays_df, players_df, player_stat_df)
    
    if len(week_data) > 0:
        all_weeks_data.append(week_data)
        print(f"Week {week} processed: {len(week_data)} plays")
    
    # Clear memory
    del tracking_df
    print(f"Week {week} completed\n")

Processing week 1...


/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).astype(bool)
/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).astype(bool)
/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasti

Week 1 processed: 498 plays
Week 1 completed

Processing week 2...


/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).astype(bool)
/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).astype(bool)
/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasti

Week 2 processed: 425 plays
Week 2 completed

Processing week 3...


...

/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).astype(bool)
/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).astype(bool)
/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasti

Week 7 processed: 478 plays
Week 7 completed

Processing week 8...


/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).astype(bool)
/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).astype(bool)
/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasti

Week 8 processed: 466 plays
Week 8 completed

Processing week 9...
Week 9 processed: 392 plays
Week 9 completed



/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).astype(bool)
/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).astype(bool)
/var/tmp/ipykernel_4248/2335167521.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasti

In [98]:
te_play_info_all = pd.concat(all_weeks_data, ignore_index=True)

In [99]:
te_play_info_all.to_parquet("te_play_info_all.parquet")

In [14]:
te_play_info_all = pd.read_parquet("te_play_info_all.parquet")

In [15]:
te_play_info_all

,nflId,displayName,y_dist,x_dist,abs_y_dist,abs_x_dist,playDirection,ball_y,gameId,playId,isDropback,passResult,playAction,offenseFormation,down,yardsToGo,absoluteYardlineNumber,pff_passCoverage,pff_runConceptPrimary,preSnapHomeScore,preSnapVisitorScore,quarter,gameClock,pff_manZone,dropbackType,routeRan,inMotionAtBallSnap,shiftSinceLineset,motionSinceLineset,score_differential,score_situation,game_phase,formation_simple,dropback_simple
0,46189.0,Will Dissly,-1.55,0.00,1.55,0.00,right,23.799999,2022091200,180,False,None,False,SINGLEBACK,2,6,68,Cover-3,INSIDE ZONE,0,0,1,12:19,Zone,None,None,False,False,False,0,close,early_game,singleback,other
1,42721.0,Eric Tomlinson,1.66,0.13,1.66,0.13,left,23.830000,2022091200,264,True,I,False,SINGLEBACK,1,10,99,Quarters,None,7,0,1,11:21,Zone,TRADITIONAL,HITCH,False,False,False,7,up_1score,early_game,singleback,traditional
2,42721.0,Eric Tomlinson,1.43,0.18,1.43,0.18,left,23.770000,2022091200,286,True,C,True,I_FORM,2,10,99,Cover-6 Right,OUTSIDE ZONE,7,0,1,11:18,Zone,DESIGNED_ROLLOUT_RIGHT,None,False,False,False,7,up_1score,early_game,i_form,rollout
3,42721.0,Eric Tomlinson,-1.60,0.30,1.60,0.30,left,29.920000,2022091200,315,False,None,False,I_FORM,1,10,74,Cover-3,OUTSIDE ZONE,7,0,1,10:26,Zone,None,None,False,False,True,7,up_1score,early_game,i_form,other
4,42721.0,Eric Tomlinson,-1.45,-0.12,1.45,0.12,left,29.809999,2022091200,346,True,C,True,SINGLEBACK,2,1,65,Cover-6 Right,INSIDE ZONE,7,0,1,09:43,Zone,TRADITIONAL,None,False,False,False,7,up_1score,early_game,singleback,traditional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150,46167.0,Jordan Akins,1.59,0.12,1.59,0.12,right,27.219999,2022110300,3369,True,IN,False,SHOTGUN,3,19,89,Quarters,None,17,29,4,02:17,Zone,TRADITIONAL,POST,False,False,False,-12,down_2scores,crunch_time,shotgun,traditional
4151,46118.0,Dallas Goedert,-1.38,-0.11,1.38,0.11,left,23.889999,2022110300,3435,True,C,True,SHOTGUN,2,12,87,Cover-1,INSIDE ZONE,17,29,4,02:00,Man,TRADITIONAL,None,False,False,False,-12,down_2scores,crunch_time,shotgun,traditional
4152,53946.0,Jack Stoll,-1.61,0.25,1.61,0.25,left,29.809999,2022110300,3538,False,None,False,I_FORM,4,1,76,Goal Line,SNEAK,17,29,4,01:36,Other,QB_SNEAK,None,False,False,False,-12,down_2scores,crunch_time,i_form,other
4153,53946.0,Jack Stoll,-1.46,-0.02,1.46,0.02,left,29.860001,2022110300,3579,False,None,False,None,1,10,75,None,UNDEFINED,17,29,4,00:55,None,UNKNOWN,None,False,False,False,-12,down_2scores,crunch_time,other,other


In [18]:
def determine_side_of_line(y_dist, play_direction):
    """
    Determine which side of the offensive line the TE is on
    
    Parameters:
    y_dist: Signed y_dist (distance from nearest tackle)
    play_direction: Direction of play ('left' or 'right')
    
    Returns:
    'left' or 'right' indicating side of offensive line
    """
    # If play is going right:
    # - positive y_dist means TE is on left side (outside left tackle)
    # - negative y_dist means TE is on right side (outside right tackle)
    if play_direction == 'right':
        return 'left' if y_dist > 0 else 'right'
    # If play is going left, it's reversed:
    else:
        return 'right' if y_dist > 0 else 'left'

In [19]:
def determine_boundary_field(row, ball_y):
    """
    Determine if TE is to the boundary or field side
    
    Args:
    row: dataframe row with side_of_line, playDirection, and ball_y
    ball_y: y-coordinate of ball at snap (0-53.3 yards)
    """
    right_hash = 23.58
    left_hash = 53.3 - 23.58  # About 29.72 yards
    tolerance = 1
    
    if row['playDirection'] == 'left':
        # Flip our hash mark interpretation when play is going left
        if abs(ball_y - right_hash) < tolerance:  # Ball on right hash (becomes field side)
            return 'boundary' if row['side_of_line'] == 'left' else 'field'
        elif abs(ball_y - left_hash) < tolerance:  # Ball on left hash (becomes boundary side)
            return 'boundary' if row['side_of_line'] == 'right' else 'field'
    else:  # playDirection == 'right'
        if abs(ball_y - right_hash) < tolerance:  # Ball on right hash (boundary side)
            return 'field' if row['side_of_line'] == 'left' else 'boundary'
        elif abs(ball_y - left_hash) < tolerance:  # Ball on left hash (field side)
            return 'field' if row['side_of_line'] == 'right' else 'boundary'
    
    return 'unknown' 

In [20]:
te_play_info_all['side_of_line'] = te_play_info_all.apply(
    lambda row: determine_side_of_line(row['y_dist'], row['playDirection']), 
    axis=1
)

te_play_info_all['boundary_field'] = te_play_info_all.apply(
    lambda row: determine_boundary_field(row, row['ball_y']), 
    axis=1
)

In [21]:
print("Side of line distribution:")
print(te_play_info_all['side_of_line'].value_counts())

Side of line distribution:
side_of_line
right    2237
left     1918
Name: count, dtype: int64


In [27]:
model_df = route_plays.copy()

In [28]:
model_df.to_parquet("model_df.parquet")